In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stopwords/stopwords.txt


In [ ]:
import os
import numpy as np
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score

# Load spaCy for text preprocessing (optional)
nlp = spacy.load("en_core_web_sm")

# Load stopwords from the provided text file
with open("stopwords.txt", "r") as file:
    stopwords = set(file.read().split())

# Step 1: Load and preprocess the data
def preprocess_text(text):
    # Preprocess text using spaCy or any other methods you prefer, and remove stopwords
    doc = nlp(text)
    preprocessed_text = " ".join([token.lemma_.lower() for token in doc if not token.is_punct and token.text.lower() not in stopwords])
    return preprocessed_text

def load_data(data_folder):
    texts = []
    labels = []
    for label in os.listdir(data_folder):
        label_folder = os.path.join(data_folder, label)
        for file_name in os.listdir(label_folder):
            file_path = os.path.join(label_folder, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
                texts.append(preprocess_text(text))
                labels.append(label)
    return texts, labels

train_folder = 'path_to_train_folder'  # Replace with the actual path to the train folder
test_folder = 'path_to_test_folder'    # Replace with the actual path to the test folder

train_texts, train_labels = load_data(train_folder)
test_texts, test_labels = load_data(test_folder)

# Step 2: Feature Extraction
vectorizer = TfidfVectorizer()  # You can also use CountVectorizer instead

X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

# Step 3: Model Training and Evaluation
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, pos_label='spam')  # Assuming 'spam' is the positive class
    return accuracy, precision

# Multinomial Naïve Bayes Classifier
naive_bayes_model = MultinomialNB()
nb_accuracy, nb_precision = train_and_evaluate_model(naive_bayes_model, X_train, train_labels, X_test, test_labels)
print("Multinomial Naïve Bayes Accuracy:", nb_accuracy)
print("Multinomial Naïve Bayes Precision:", nb_precision)

# Logistic Regression Classifier
logistic_regression_model = LogisticRegression(max_iter=1000)  # Adjust max_iter as needed
lr_accuracy, lr_precision = train_and_evaluate_model(logistic_regression_model, X_train, train_labels, X_test, test_labels)
print("Logistic Regression Accuracy:", lr_accuracy)
print("Logistic Regression Precision:", lr_precision)
